In [46]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px
import torch

from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [47]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
embedder = SentenceTransformer('all-MiniLM-L6-v2')

data = dict()
# folders = ['all_gpt_summaries', 'all_philosophize_this_transcripts', 'all_source_texts', 'all_wikipedias']
folders = ['all_gpt_summaries']
# folders = ['select_all_texts']

for folder in folders:
    data_folder = os.path.join('data', 'all_texts', folder)
    for filename in os.listdir(data_folder):
        filepath = os.path.join(data_folder, filename)
        tensors = []

        with open(filepath, 'r', encoding='utf-8') as file:
            text = file.read()
        chunks = splitter.split_text(text)
        embeddings = embedder.encode(chunks)

        for embedding in embeddings:
            tensors.append(torch.tensor(embedding))

        tensor_stack = torch.stack(tensors, dim=1)
        mean_tensor = tensor_stack.mean(dim=1)
        data[filename] = mean_tensor

In [48]:
filename_to_category = {
    "aurelius_meditations.txt": "Stoicism",
    "beauvoir_ethicsambiguity.txt": "Existentialism",
    "buddha_stanford_pe.txt": "Buddhism",
    "camus_sisyphus.txt": "Absurdism",
    "camus_thestranger.txt": "Absurdism",
    "confucius_analects.txt": "Confucianism",
    "emerson_selfreliance.txt": "Transcendentalism",
    "epicurus_fragments.txt": "Epicureanism",
    "epicurus_lettermenoeceus.txt": "Epicureanism",
    "essence_of_buddha.txt": "Buddhism",
    "kafka_thetrial.txt": "Absurdism",
    "kierkegaard_presentage.txt": "Existentialism",
    "laozi_taoteching.txt": "Taoism",
    "leibniz_theodicity.txt": "Rationalism",
    "nietzsche_tsz.txt": "Existentialism",
    "spinoza_ethics.txt": "Rationalism",
    "thoreau_walden.txt": "Transcendentalism",
    "absurdism_wikipedia.txt": "Absurdism",
    "buddhism_wikipedia.txt": "Buddhism",
    "confucianism_wikipedia.txt": "Confucianism",
    "effectivealtruism_wikipedia.txt": "Effective Altruism",
    "epicureanism_wikipedia.txt": "Epicureanism",
    "existentialism_wikipedia.txt": "Existentialism",
    "rationalism_wikipedia.txt": "Rationalism",
    "stoicism_wikipedia.txt": "Stoicism",
    "taoism_wikipedia.txt": "Taoism",
    "transcendentalism_wikipedia.txt": "Transcendentalism",
    "gpt_beauvoir.txt": "Existentialism",
    "gpt_buddhism.txt": "Buddhism",
    "gpt_camus.txt": "Absurdism",
    "gpt_emerson.txt": "Transcendentalism",
    "gpt_epicureanism.txt": "Epicureanism",
    "gpt_kafka.txt": "Absurdism",
    "gpt_kierkegaard.txt": "Existentialism",
    "gpt_leibniz.txt": "Rationalism",
    "gpt_nietzsche.txt": "Existentialism",
    "gpt_spinoza.txt": "Rationalism",
    "gpt_stoicism.txt": "Stoicism",
    "gpt_thoreau.txt": "Transcendentalism",
    "philthis_beauvoir.txt": "Existentialism",
    "philthis_buddhism.txt": "Buddhism",
    "philthis_camus.txt": "Absurdism",
    "philthis_emerson.txt": "Transcendentalism",
    "philthis_epicurus.txt": "Epicureanism",
    "philthis_kierkegaard.txt": "Existentialism",
    "philthis_leibniz.txt": "Rationalism",
    "philthis_nietzsche.txt": "Existentialism",
    "philthis_spinoza.txt": "Rationalism",
    "philthis_stoicism.txt": "Stoicism",
    "philthis_thoreau.txt": "Transcendentalism",
}

In [49]:
def plot_pca_3d(data, filename_to_category, title="PCA 3D Plot"):
    """
    Conducts PCA on a list of tensor vectors and plots the result in a 3D interactive chart.
    
    Args:
        data (dict): A dictionary with 'filename: tensor' pairs.
        filename_to_category (dict): A dictionary mapping filenames to categories (labels).
        title (str): The title of the plot.
    """
    # Aggregate tensors and metadata
    filenames = []
    categories = []
    tensors = []
    
    for filename, tensor in data.items():
        filenames.append(filename)
        categories.append(filename_to_category[filename])
        tensors.append(tensor.numpy())  # Convert to NumPy array for PCA

    # Stack tensors into a matrix for PCA
    tensor_matrix = np.stack(tensors)  # Shape: [num_files, 384]

    # Perform PCA to reduce dimensions to 3
    pca = PCA(n_components=3)
    pca_result = pca.fit_transform(tensor_matrix)  # Shape: [num_files, 3]

    # Create a DataFrame for plotting
    import pandas as pd
    plot_data = pd.DataFrame({
        'PC1': pca_result[:, 0],
        'PC2': pca_result[:, 1],
        'PC3': pca_result[:, 2],
        'Filename': filenames,
        'Category': categories
    })

    # Plot using Plotly for interactivity
    fig = px.scatter_3d(
        plot_data,
        x='PC1',
        y='PC2',
        z='PC3',
        color='Category',  # Color by category
        hover_name='Filename',  # Show filename on hover
        title=title
    )
    
    fig.update_traces(marker=dict(size=5))  # Adjust marker size
    fig.update_layout(
        scene=dict(
            xaxis_title='PC 1',
            yaxis_title='PC 2',
            zaxis_title='PC 3'
        ),
        width=1200,  # Set plot width
        height=800   # Set plot height
    )

    fig.show()


plot_pca_3d(data, filename_to_category, title="PCA Visualization of Source Texts")

In [51]:
def plot_pca_2d(data, filename_to_category, title="PCA 2D Plot"):
    """
    Conducts PCA on a list of tensor vectors and plots the result in a 2D interactive chart.
    
    Args:
        data (dict): A dictionary with 'filename: tensor' pairs.
        filename_to_category (dict): A dictionary mapping filenames to categories (labels).
        title (str): The title of the plot.
    """
    # Aggregate tensors and metadata
    filenames = []
    categories = []
    tensors = []
    
    for filename, tensor in data.items():
        filenames.append(filename)
        categories.append(filename_to_category[filename])
        tensors.append(tensor.numpy())  # Convert to NumPy array for PCA

    # Stack tensors into a matrix for PCA
    tensor_matrix = np.stack(tensors)  # Shape: [num_files, 384]

    # Perform PCA to reduce dimensions to 2
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(tensor_matrix)  # Shape: [num_files, 2]

    # Create a DataFrame for plotting
    import pandas as pd
    plot_data = pd.DataFrame({
        'PC1': pca_result[:, 0],
        'PC2': pca_result[:, 1],
        'Filename': filenames,
        'Category': categories
    })

    # Plot using Plotly for interactivity
    fig = px.scatter(
        plot_data,
        x='PC1',
        y='PC2',
        color='Category',  # Color by category
        hover_name='Filename',  # Show filename on hover
        title=title
    )
    
    fig.update_traces(marker=dict(size=7))  # Adjust marker size
    fig.update_layout(
        xaxis_title='Principal Component 1',
        yaxis_title='Principal Component 2',
        width=1000,  # Set plot width
        height=800   # Set plot height
    )

    fig.show()

plot_pca_2d(data, filename_to_category, title="PCA Visualization of GPT Summaries (2D)")